<div class="alert alert-danger">
    <h4 style="font-weight: bold; font-size: 28px;">NBA API</h4>
    <p style="font-size: 20px;">Data Gathering</p>
</div>

<a name="NBA"></a>

# Setup

In [3]:
import pandas as pd
from datetime import datetime, timedelta
import time

In [4]:
from nba_api.stats.static import teams, players
from nba_api.stats.endpoints import (
  scoreboard, leaguegamefinder, playercareerstats,
  boxscorematchupsv3, boxscoreadvancedv2, teamestimatedmetrics, 
  teamgamelogs, TeamGameLogs, TeamEstimatedMetrics, leaguedashteamstats,
    hustlestatsboxscore, boxscoremiscv2
)

# Team Data

In [6]:
# get_teams returns a list of 30 dictionaries, each an NBA team
nba_teams = teams.get_teams()
print("Number of teams fetched: {}".format(len(nba_teams)))
nba_teams_df = pd.DataFrame(nba_teams)
nba_teams_df.head()

Number of teams fetched: 30


,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Georgia,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966


# Players

In [7]:
# get_players returns a list of dictionaries, each representing a player
nba_players = players.get_players()
print("Number of players fetched: {}".format(len(nba_players)))
nba_players_df = pd.DataFrame(nba_players)
nba_players_df.head()

Number of players fetched: 4900


,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False


# Scoreboard

In [ ]:
# Today's Score Board
games = scoreboard.ScoreBoard()

# json
games.get_json()

# dictionary
games.get_dict()

# League Game Finder

In [9]:
# get game data
team_ids = nba_teams_df['id'].tolist()

games_list = []

for id in team_ids:
    print(id)
    # query for games
    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=id)
    # we want the first DataFrame of those returned
    games_list.append(gamefinder.get_data_frames()[0])
    # add time delay between requests
    time.sleep(3)

1610612737
1610612738
1610612739
1610612740
1610612741
1610612742
1610612743
1610612744
1610612745
1610612746
1610612747
1610612748
1610612749
1610612750
1610612751
1610612752
1610612753
1610612754
1610612755
1610612756
1610612757
1610612758
1610612759
1610612760
1610612761
1610612762
1610612763
1610612764
1610612765
1610612766


In [10]:
games_df = pd.concat(games_list)
print(games_df.shape)
games_df.head()

(104884, 28)


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22023,1610612737,ATL,Atlanta Hawks,0022300821,2024-02-25,ATL vs. ORL,W,241,109,...,0.882,9.0,35.0,44.0,31,10.0,3,9,17,17.0
1,22023,1610612737,ATL,Atlanta Hawks,0022300804,2024-02-23,ATL vs. TOR,L,239,121,...,0.708,20.0,34.0,54.0,27,5.0,4,9,11,-2.0
2,22023,1610612737,ATL,Atlanta Hawks,0022300777,2024-02-14,ATL @ CHA,L,239,99,...,0.900,15.0,30.0,45.0,23,5.0,6,16,20,-23.0
3,22023,1610612737,ATL,Atlanta Hawks,0022300763,2024-02-12,ATL vs. CHI,L,241,126,...,0.828,15.0,34.0,49.0,33,4.0,3,11,20,-10.0
4,22023,1610612737,ATL,Atlanta Hawks,0022300754,2024-02-10,ATL vs. HOU,W,240,122,...,0.757,9.0,32.0,41.0,25,7.0,8,11,18,9.0


In [11]:
games_df['GAME_DATE'] = pd.to_datetime(games_df['GAME_DATE'])

# get the latest date
earliest_date = games_df['GAME_DATE'].max()
print(earliest_date)

2024-02-25 00:00:00


In [ ]:
games_df.to_csv('../data/original/nba_games_box_scores_1984_2024.csv', index=False)

# League Dash Team Stats

In [ ]:
ldts = leaguedashteamstats.LeagueDashTeamStats(month=5, season='2023-24')

In [ ]:
ldts.get_data_frames()[0].head()

In [ ]:
ldts_list = []
seasons = []
for year in range(1996, 2024):
    season = f"{year}-{str(year + 1)[-2:]}"
    seasons.append(season)
months = range(1, 13)

for season in seasons:
    for month in months:
        print(f"Querying season {season}, month {month}")
        # query for months
        ldts = leaguedashteamstats.LeagueDashTeamStats(month=month, season=season)
        # get the first DataFrame of those returned
        df = ldts.get_data_frames()[0]
        
        # add columns for 'season' and 'month'
        df['SEASON'] = season
        df['MONTH'] = month
        
        # append the DataFrame to the list
        ldts_list.append(df)
        
        # add time delay between requests
        time.sleep(3)

# concatenate all DataFrames in the list into one large DataFrame
ldts_df = pd.concat(ldts_list, ignore_index=True)

In [ ]:
ldts_df.head()

In [ ]:
ldts_df.to_csv('../data/original/nba_dash_team_stats_1997_2024.csv', index=False)

In [ ]:
ldts_df.value_counts('TEAM_NAME')

In [ ]:
BC_2023 = ldts_df[(ldts_df['TEAM_NAME'] == 'Boston Celtics') & (ldts_df['SEASON'] == '2022-23')]
BC_2023.head()

# Player Career Statistics

In [ ]:
# get player data
player_ids = nba_players_df['id'].tolist()

players_stats_list = []

for id in player_ids:
    print(id)
    # query for games
    career = playercareerstats.PlayerCareerStats(player_id=id)
    # we want the first DataFrame of those returned
    players_stats_list.append(career.get_data_frames()[0])
    # add time delay between requests
    time.sleep(1)

In [ ]:
players_stats_df = pd.concat(players_stats_list)
print(players_stats_df.shape)
players_stats_df.head()

In [ ]:
# get the latest season
earliest_date = players_stats_df['SEASON_ID'].min()
print(earliest_date)

In [ ]:
players_stats_df.to_csv('../data/original/nba_players_statistics_1946_2024.csv', index=False)

# Box Score Matchups V3

In [ ]:
games_df = pd.read_csv('../data/original/nba_games_box_scores_1984_2024.csv')
# get player data
game_ids = games_df['GAME_ID'].tolist()

In [ ]:
game_ids[5000]

In [ ]:
bs_matchups = boxscorematchupsv3.BoxScoreMatchupsV3(game_id=21000400)
#bs_matchups.get_data_frames()

# Box Score Advanced V3

In [ ]:
bs_adv = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id=21000400)
check = bs_adv.get_data_frames()[1]
check

In [16]:
# function to get game_ids
def get_game_ids(season_id):
    game_ids = []
    game_ids = games_df['GAME_ID'][(games_df.SEASON_ID == season_id)].tolist()
    return game_ids

#get game ids for 2023 - 2024 season
game_ids_2023_2024 = get_game_ids('22023')

#get game ids for 2022 - 2023 season
game_ids_2022_2023 = get_game_ids('22022')

#get game ids for 2021 - 2022 season
game_ids_2021_2022 = get_game_ids('22021')

In [ ]:
# function to get team advanced stats per game for a given season
def get_adv_stats_df(game_id_list):
    adv_games_stats_list = []
    for id in game_id_list:
        print(id)
        # query for games
        games = boxscoreadvancedv3.BoxScoreAdvancedV3(game_id=id)
        adv_games_stats_list.append(games.get_data_frames()[1])
        time.sleep(3)
    adv_stats_df = pd.concat(adv_games_stats_list, ignore_index=True)
    adv_stats_df = adv_stats_df.drop_duplicates()
    return adv_stats_df

In [ ]:
# get advanced stats df for 2021 - 2022 season
adv_stats_df_2021_2022 = get_adv_stats_df(game_ids_2021_2022)
adv_stats_df_2021_2022.head()

In [ ]:
# get advanced stats df for 2022 - 2023 season
adv_stats_df_2022_2023 = get_adv_stats_df(game_ids_2022_2023)
adv_stats_df_2022_2023.head()

In [ ]:
# get advanced stats df for 2023 - 2024 season
adv_stats_df_2023_2024 = get_adv_stats_df(game_ids_2023_2024)
adv_stats_df_2023_2024.head()

In [ ]:
#combine advanced stat dataframes into one combined datafram
adv_stats_frames = [adv_stats_df_2021_2022, adv_stats_df_2022_2023, adv_stats_df_2023_2024]
adv_stats_df = pd.concat(adv_stat_frames)
adv_stats_df.head()

In [ ]:
#creates team name field to match games_df
adv_stats_df['TEAM_NAME'] = adv_stats_df['teamCity'] + " " + adv_stats_df['teamName']

#renames fields that match games_df
adv_stats_df.rename(columns={'gameId':'GAME_ID','teamId':'TEAM_ID', 'teamTricode': 'TEAM_ABBREVIATION'}, inplace=True)

#drop redundant columns
adv_stats_df.drop(['teamCity', 'teamName', 'teamSlug'], inplace=True, axis=1)

adv_stats_df.head()

In [ ]:
#get GAME_DATE, MATCHUP, GAME_ID, TEAM_ABBREVIATION fields from games_df
adv_stats_df = pd.merge(adv_stats_df, games_df[['SEASON_ID','GAME_DATE','MATCHUP', 'GAME_ID','TEAM_ID', 'TEAM_ABBREVIATION']], on=['GAME_ID','TEAM_ID', 'TEAM_ABBREVIATION'])

adv_stats_df.head()

In [ ]:
#export csv
adv_stats_df.to_csv('../data/original/nba_advanced_statistics_2021_2024.csv', index=False)

# Team Game Logs

In [ ]:
mav_id = '1610612742'
logs = teamgamelogs.TeamGameLogs.DataSet(data=)
#logs.get_data_frame(data=)

# Team Estimated Metrics

In [ ]:
team_metrics = teamestimatedmetrics.TeamEstimatedMetrics.DataSet(data=)

In [ ]:
from nba_api.stats.endpoints import commonplayerinfo

# Basic Request
player_info = commonplayerinfo.CommonPlayerInfo(player_id=2544)

In [ ]:
player_info.available_seasons.get_data_frame()

# Team Hustle Stats

In [12]:
hustle = hustlestatsboxscore.HustleStatsBoxScore(game_id = "0022100209")
hustle.get_data_frames()[2]

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MINUTES,PTS,CONTESTED_SHOTS,CONTESTED_SHOTS_2PT,CONTESTED_SHOTS_3PT,...,SCREEN_ASSISTS,SCREEN_AST_PTS,OFF_LOOSE_BALLS_RECOVERED,DEF_LOOSE_BALLS_RECOVERED,LOOSE_BALLS_RECOVERED,OFF_BOXOUTS,DEF_BOXOUTS,BOX_OUT_PLAYER_TEAM_REBS,BOX_OUT_PLAYER_REBS,BOX_OUTS
0,0022100209,1610612741,Bulls,CHI,Chicago,240.000000:00,121,45,21,24,...,9,20,2,3,5,1,2,3,0,3
1,0022100209,1610612747,Lakers,LAL,Los Angeles,240.000000:00,103,53,30,23,...,6,13,1,4,5,1,1,2,2,2


In [24]:
# function to get team hustle stats per game for a given season
def get_hustle_stats_df(game_id_list):
    hustle_games_stats_list = []
    for id in game_id_list:
        print(id)
        # query for games
        hustle = hustlestatsboxscore.HustleStatsBoxScore(game_id=id)
        hustle_games_stats_list.append(hustle.get_data_frames()[2])
        time.sleep(1)
    hustle_stats_df = pd.concat(hustle_games_stats_list, ignore_index=True)
    hustle_stats_df = hustle_stats_df.drop_duplicates()
    return hustle_stats_df

In [25]:
# get hustle stats df for 2021 - 2022 season
hustle_stats_df_2021_2022 = get_hustle_stats_df(game_ids_2021_2022)
hustle_stats_df_2021_2022.head()

0022101221


ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [ ]:
# get hustle stats df for 2022 - 2023 season
hustle_stats_df_2022_2023 = get_hustle_stats_df(game_ids_2022_2023)
hustle_stats_df_2022_2023.head()

In [ ]:
# get hustle stats df for 2023 - 2024 season
hustle_stats_df_2023_2024 = get_hustle_stats_df(game_ids_2023_2024)
hustle_stats_df_2023_2024.head()

In [ ]:
#combine hustle stat dataframes into one combined dataframe
hustle_stats_frames = [hustle_stats_df_2021_2022, hustle_stats_df_2022_2023, hustle_stats_df_2023_2024]
hustle_stats_df = pd.concat(hustle_stats_frames)
hustle_stats_df.head()

In [ ]:
#creates team name field to match games_df
hustle_stats_df['TEAM_NAME'] = hustle_stats_df['TEAM_CITY'] + " " + hustle_stats_df['TEAM_NAME']

#renames fields that match games_df
#hustle_stats_df.rename(columns={'gameId':'GAME_ID','teamId':'TEAM_ID', 'teamTricode': 'TEAM_ABBREVIATION'}, inplace=True)

#drop redundant columns
hustle_stats_df.drop(['TEAM_CITY'], inplace=True, axis=1)

hustle_stats_df.head()

In [ ]:
hustle_stats_df = pd.merge(hustle_stats_df, games_df[['SEASON_ID','GAME_DATE','MATCHUP', 'GAME_ID','TEAM_ID', 'TEAM_ABBREVIATION']], on=['GAME_ID','TEAM_ID', 'TEAM_ABBREVIATION'])

hustle_stats_df.head()

In [ ]:
#export csv
hustle_stats_df.to_csv('../data/original/nba_hustle_statistics_2021_2024.csv', index=False)

# Box Score Misc

In [ ]:
misc = boxscoremiscv2.BoxScoreMiscV2(game_id = "0022100209")
misc.get_data_frames()[1]

In [26]:
# function to get team hustle stats per game for a given season
def get_misc_stats_df(game_id_list):
    misc_games_stats_list = []
    for id in game_id_list:
        print(id)
        # query for games
        misc = boxscoremiscv2.BoxScoreMiscV2(game_id=id)
        misc_games_stats_list.append(misc.get_data_frames()[1])
        time.sleep(1)
    misc_stats_df = pd.concat(misc_games_stats_list, ignore_index=True)
    misc_stats_df = misc_stats_df.drop_duplicates()
    return misc_stats_df

In [27]:
# get miscellaneous stats df for 2021 - 2022 season
misc_stats_df_2021_2022 = get_misc_stats_df(game_ids_2021_2022)
misc_stats_df_2021_2022.head()

0022101221
0022101207
0022101192
0022101182
0022101163
0022100452
0022101143
0022101124


ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [28]:
# get miscellaneous stats df for 2022 - 2023 season
misc_stats_df_2022_2023 = get_misc_stats_df(game_ids_2022_2023)
misc_stats_df_2022_2023.head()

0022201216
0022201205
0022201191


ReadTimeout: HTTPSConnectionPool(host='stats.nba.com', port=443): Read timed out. (read timeout=30)

In [ ]:
# get miscellaneous stats df for 2023 - 2024 season
misc_stats_df_2023_2024 = get_misc_stats_df(game_ids_2023_2024)
misc_stats_df_2023_2024.head()

In [ ]:
#combine hustle stat dataframes into one combined dataframe
misc_stats_frames = [misc_stats_df_2021_2022, misc_stats_df_2022_2023, misc_stats_df_2023_2024]
misc_stats_df = pd.concat(misc_stats_frames)
misc_stats_df.head()

In [ ]:
#creates team name field to match games_df
misc_stats_df['TEAM_NAME'] = misc_stats_df['TEAM_CITY'] + " " + misc_stats_df['TEAM_NAME']

#renames fields that match games_df
#hustle_stats_df.rename(columns={'gameId':'GAME_ID','teamId':'TEAM_ID', 'teamTricode': 'TEAM_ABBREVIATION'}, inplace=True)

#drop redundant columns
misc_stats_df.drop(['TEAM_CITY'], inplace=True, axis=1)

misc_stats_df.head()

In [ ]:
misc_stats_df = pd.merge(misc_stats_df, games_df[['SEASON_ID','GAME_DATE','MATCHUP', 'GAME_ID','TEAM_ID', 'TEAM_ABBREVIATION']], on=['GAME_ID','TEAM_ID', 'TEAM_ABBREVIATION'])

misc_stats_df.head()

In [ ]:
#export csv
misc_stats_df.to_csv('../data/original/nba_misc_boxscore_statistics_2021_2024.csv', index=False)